___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Microestructuras y Sistema de Trading
## Laboratorio 1

### EQUIPO:
#### - Tere
#### - Ana Laura Correa
#### - Gaspar Benitez
#### - Marysol Cantarero

##### Situación: 
Acabas de entrar a trabajar a una operadora de fondos de inversión como analista financiero dentro de la mesa de análisis del mercado de renta variable. En la junta trimestral se plantea la posibilidad de crear un fondo propio con exposición al mercado de valores de USA tratando de replicar a los ETFs conocidos como ishares que la empresa BlackRock ofrece. 

La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.


##### Características mínimas de solución: 
Debido a tu formación, te piden que este proceso que construyas sea repetible para cualquier ETF y también incluir 3 medidas de atribución al desempeño, tanto para el caso que el ETF nunca hubiese sido rebalanceado como para los casos donde sí. 

En la mesa de análisis están interesados en conocer el proceso que sigues para adquirir, limpiar, dar formato y utilizar los datos, y a pesar que el equipo senior de análisis está compuesto por personal altamente calificado en temas de economía y finanzas, ellos no saben programar por lo que te piden que incluyas el código utilizado, y que sea de manera ordenada y explicando el sentido de las funciones y paqueterías que requeriste. 

Comparar con 1 año de información: 1.3.17 al 1.3.18 

Utilizar un ETF de Ishares para el mercado de renta varialbe de USA 

Entregar un Jupyter Notebook con explicación y comentarios en el código 

Crear y actualizar continuamente un repositorio en GitHub para el proyecto.

### RESOLUCIÓN DE PROBLEMA

Los ETF(Exchange Trade Funds) son fondos o instrumentos de inversión cuyos títulos cotizan en la bolsa, al igual que una acción bursátil,pudiendose comprar y vender en cualquier momento, sin necesidad de esperar al cierre del mercado. Su objetivo es replicar el comportamiento de los activos que componen un índice bursátil, ya sea de renta fija, variable, divisas o materias primas u otros activos financieros.

Para este problema se analizara el ETF "iShares U.S. Financials ETF (IYF)" para el mercado de renta variable de USA con posiciones en directo tomando información del 01/Agosto/2018 al 01/Agosto/2019. Sin embargo, el código a realizar debe ser repetible para cualquier ETF.

Las 3 medidas de atribución al desempeño que se eligieron para la comparación de los resultados son: Rendimiento, volatilidad y medidas de desempeño.

In [1]:
# DESCARGA DE LIBRERIAS
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH"  # API key Quandl
fechas = list(['2017-02-28', '2017-03-31', '2017-04-28', '2017-05-31', '2017-06-30',
                 '2017-07-31', '2017-08-31', '2017-09-29', '2017-10-31', '2017-11-30',
                 '2017-12-29', '2018-01-31', '2018-02-28'])
narch = 13

### DESCARGA DE DATOS

In [3]:
# FUNCIÓN PARA DESCARGA DE ARCHIVOS CON DATOS HOLDINGS ETF (ISHARES)
def holdings_csv(narch,name):
    data = []
    for i in range(narch):
        r = name+str(i)+".csv"
        info = pd.read_csv(r,header=10, index_col= 0)
        info = info.dropna() # quitando los datos vacios.
        info = info.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
        data.append(info)
    return data
holdings = holdings_csv(narch,"IYF_holdings")

In [142]:
# FUNCIÓN PARA DESCARGA DE DATOS HISTÓRICOS (QUANDL)
def download_quandl(fechas, narch):
    d2 = []  # Empty dictionary
    for i in range(narch):  # Loop to Download data from Quandl
        index= holdings[i].index
        q, h = holdings[i].shape
        ticker_r = list(index)  # ticker = 'WIKI/AAPL'
        date = {'gte': fechas[i], 'lte': fechas[i] }
        column = {'columns': ['ticker', 'date', 'Adj_close']}
        data = quandl.get_table('WIKI/PRICES', qopts=column, ticker=ticker_r, date=date)
        data = data.set_index('ticker')
        d2.append(data)
    return d2
prices = download_quandl(fechas,narch)

In [8]:
# NAV (Net Asset Value)
nav = pd.read_excel('nav.xlsx', sheetname="Historical", index_col=0)['NAV per Share']

C:\Users\marisol\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [468]:
# LIMPIEZA DE DATOS 
def clean_data(narch, prices, holdings):
    d =[]
    hold = []
    for i in range(narch):
        a = set(list(prices[i].index))
        b = set(list(holdings[i].index))
        c = []
        for item in b:
            if item not in a:
                 c.append(item)
        d.append(c)
        holdi = holdings[i].drop(d[i]) # Se eliminan datos de ishares que no estan en quandl.
        hold.append(holdi)
    return hold
d = clean_data(narch, prices, holdings)
d

[        Weight (%)   Price        Shares
 Ticker                                  
 JPM           6.83   90.62  1,591,119.00
 WFC           5.51   57.88  2,009,905.00
 BAC           5.26   24.68  4,493,330.00
 C             3.59   59.81  1,267,168.00
 V             3.46   87.94    830,443.00
 MA            2.22  110.46    423,253.00
 GS            1.93  248.06    164,442.00
 USB           1.85   55.00    710,437.00
 MS            1.39   45.67    641,235.00
 CB            1.35  138.17    206,919.00
 AIG           1.31   63.92    433,892.00
 PNC           1.31  127.23    216,637.00
 AXP           1.30   80.06    341,862.00
 SPG           1.22  184.40    139,728.00
 MET           1.21   52.44    488,744.00
 BK            1.05   47.14    470,158.00
 AMT           1.03  114.79    189,288.00
 SCHW          1.03   40.41    536,424.00
 PRU           1.00  110.54    191,204.00
 BLK           0.99  387.46     54,129.00
 COF           0.95   93.86    214,461.00
 CME           0.87  121.46    150

##### Lo unico que se moverá son los pesos. price y shares quedan igual.
### CASO 1 : CONSTRUCCIÓN DE PORTAFOLIO DE INVERSIÓN SIN REBALANCEO (mismos precios).

In [145]:
w = holdings[0]['Weight (%)'] # mismos pesos
p = holdings[0]['Price']   # precios
s = holdings[0]['Shares']  #número de acciones
string = []
for j in s:
    st= float(j.replace(',',''))
    string.append(st)
a1 = w*p # Valor en dinero de cada acción
v_etf1 = sum(a1) # Valor Total ETF
etf1.append(v_etf1)

In [110]:
w = holdings[0]['Weight (%)'] # mismos pesos
etf1 = []
for i in range(narch):
    p = holdings[i]['Price']   # precios
    s = holdings[i]['Shares']  #número de acciones
    string = []
    for j in s:
        st= float(j.replace(',',''))
        string.append(st)
    a1 = w*p # Valor en dinero de cada acción
    v_etf1 = sum(a1) # Valor Total ETF
    etf1.append(v_etf1)

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:

#a = p*w*s # Valor en dinero de cada acción
#v_etf_total = sum(a) #Valor del portafolio o etf, suma de los valores de las acciones

In [ ]:
# Desviación estandar
nav = pd.read_excel('iShares-US-Financials-ETF_fund.xls',sheet_name=1, index_col= 0)
nav = nav["NAV per Share"][11:263]
#nav = list(nav)
#nav.reverse()
desv1 = np.std(nav)
mean1 = np.mean(nav)

### CRITERIO DE RE BALANCEO.

In [ ]:
#SERIE DE TIEMPO HACIENDO RE BALANCEO
plt.title("Serie de tiempo ETF IYF")
plt.ylabel("NAV")
plt.xlabel("Fechas")
plt.axvline(x=0, c = "green")
plt.axvline(x=22, c = "greenyellow")
plt.axvline(x=43, c = "tomato")
plt.axvline(x=64, c = "lightcoral")
plt.axvline(x=85, c = "aqua")
plt.axvline(x=106, c = "plum")
plt.axvline(x=125, c = "mediumvioletred")
plt.axvline(x=147, c = "lawngreen")
plt.axvline(x=166, c = "greenyellow")
plt.axvline(x=186, c = "gold")
plt.axvline(x=209, c = "teal")
plt.axvline(x=229, c = "lavender")
plt.axvline(x=251, c = "darkmagenta")
plt.plot(nav, c = "k")

### COMPARACIÓN DE RESULTADOS: REBALANCEO VS. MISMAS POSICIONES

In [ ]:
# MEDIDAS DE ATRIBUCIÓN AL DESEMPEÑO

### CONCLUSIONES
.....

### BIBLIOGRAFÍA
https://www.portafolio.co/economia/finanzas/son-etf-293754

https://economipedia.com/definiciones/etf-fondos-cotizados.html

https://www.ishares.com/us/products/239508/#/

https://www.quandl.com/tools/python

